In [1]:
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
!wget https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/noisy_test_data.txt

--2025-03-18 16:13:43--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/train_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375849 (367K) [text/plain]
Saving to: ‘train_data.txt’

train_data.txt      100%[===================>] 367.04K  --.-KB/s    in 0.04s   

2025-03-18 16:13:43 (8.08 MB/s) - ‘train_data.txt’ saved [375849/375849]

--2025-03-18 16:13:44--  https://raw.githubusercontent.com/debajyotimaz/nlp_assignment/refs/heads/main/Viterbi_assignment/test_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP reques

In [2]:
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            sentence = []
            for token in line.strip().split():
                word, tag = token.rsplit('/', 1)  # Split word and tag
                sentence.append((word, tag))
            data.append(sentence)
    return data

# Load train and test data from files
train_data_file = '/content/train_data.txt'  # Path to your training data file
test_data_file = '/content/test_data.txt'    # Path to your test data file
noisy_test_data_file = '/content/noisy_test_data.txt'  # Path to your noisy test data file

train_data = load_data(train_data_file)
test_data = load_data(test_data_file)
noisy_test_data = load_data(noisy_test_data_file)

# Print a sample from the training data
print(train_data[0])

[('He', 'PRON'), ('let', 'VERB'), ('her', 'PRON'), ('tell', 'VERB'), ('him', 'PRON'), ('all', 'PRT'), ('about', 'ADP'), ('the', 'DET'), ('church', 'NOUN'), ('.', '.')]


In [3]:
# Start Probabilities
# Transition Probabilities
# Emission Probabilities

# Start Probabilites:
#   First find all the unique pos tags
#   Calculate P(pos tag at the start of sentence):
#       Ex: P(PRON|START) = No. of Pron at train_data[i][0][1]/No. of train_data

start_tags = [sentence[0][1] for sentence in train_data]
# print(start_tags)
# print(set(start_tags))

start_probabilities = {}
for tag in set(start_tags):
  start_probabilities[tag] = start_tags.count(tag)/len(start_tags)

print("Start Probabilites: ",start_probabilities)


# print(len(train_data))


Start Probabilites:  {'ADP': 0.1345, 'DET': 0.201, 'ADJ': 0.0365, 'ADV': 0.0965, '.': 0.083, 'VERB': 0.0485, 'NUM': 0.016, 'X': 0.0005, 'CONJ': 0.053, 'PRT': 0.031, 'NOUN': 0.1505, 'PRON': 0.149}


In [4]:
from collections import Counter, defaultdict
# Transition Probabilites:
#   Transition Probability P(t_j|t_i) = Count(t_i,t_j)/Count(t_i)
#      Ex: P(VERB|NOUN) = Count(NOUN VERB)/Count(NOUN)
tags = [word[1] for sentence in train_data for word in sentence]
# print(tags)
unique_tags = [i for i in set(tags) if i != 'X']

transition_counts = defaultdict(Counter)
for sentence in train_data:
  for i in range(len(sentence)-1):
    current_tag = sentence[i][1]
    next_tag = sentence[i+1][1]
    transition_counts[current_tag][next_tag] += 1

print(transition_counts)
print(transition_counts['ADJ']['DET'])
print(sum(transition_counts['PRON'].values()))

transition_probabilities = defaultdict(Counter)
for next_tag in unique_tags:
  for current_tag in unique_tags:
    transition_probabilities[next_tag][current_tag] = (transition_counts[current_tag][next_tag]+1)/sum(transition_counts[current_tag].values())

print(transition_probabilities)




defaultdict(<class 'collections.Counter'>, {'PRON': Counter({'VERB': 1193, '.': 182, 'ADP': 91, 'ADV': 88, 'PRT': 32, 'DET': 28, 'CONJ': 27, 'NOUN': 15, 'ADJ': 14, 'PRON': 14, 'NUM': 1}), 'VERB': Counter({'VERB': 1138, 'ADP': 1093, 'DET': 1009, 'ADV': 648, 'NOUN': 615, '.': 474, 'ADJ': 365, 'PRT': 361, 'PRON': 324, 'CONJ': 103, 'NUM': 47, 'X': 1}), 'PRT': Counter({'VERB': 616, 'ADP': 76, 'DET': 73, '.': 59, 'NOUN': 32, 'ADV': 26, 'ADJ': 23, 'PRT': 9, 'PRON': 9, 'CONJ': 5, 'NUM': 3}), 'ADP': Counter({'DET': 2291, 'NOUN': 1315, 'ADJ': 420, 'PRON': 367, 'VERB': 206, 'NUM': 157, 'ADP': 116, 'ADV': 72, 'PRT': 60, '.': 40, 'CONJ': 18}), 'DET': Counter({'NOUN': 2891, 'ADJ': 1137, 'VERB': 262, 'ADV': 76, '.': 65, 'PRON': 47, 'ADP': 45, 'NUM': 44, 'DET': 31, 'PRT': 14, 'X': 10, 'CONJ': 4}), 'NOUN': Counter({'.': 2659, 'ADP': 2373, 'VERB': 1482, 'NOUN': 1344, 'CONJ': 571, 'ADV': 236, 'PRON': 196, 'PRT': 175, 'ADJ': 120, 'DET': 119, 'NUM': 67, 'X': 8}), 'ADV': Counter({'VERB': 446, '.': 310, 'ADP

In [5]:
print(unique_tags)

['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB']


In [6]:
# Emission Probabilities
#   Find unique words
#   P(word|pos) = Count(pos,word)/Count(pos)

words = [word[0].lower() for sentence in train_data for word in sentence]

emission_counts = defaultdict(Counter)
for sentence in train_data:
  for i in range(len(sentence)-1):
    current_tag = sentence[i][1]
    current_word = sentence[i][0]
    emission_counts[current_tag][current_word] += 1

print(emission_counts)
print(emission_counts['PRON']['he'])
print(sum(emission_counts['PRON'].values()))

emission_probabilities = defaultdict(Counter)
for current_word in set(words):
  for current_tag in unique_tags:
    emission_probabilities[current_word][current_tag] = (emission_counts[current_tag][current_word]+1)/sum(emission_counts[current_tag].values())

print(emission_probabilities)
print(emission_probabilities['College'])
# print(set(tags))


defaultdict(<class 'collections.Counter'>, {'PRON': Counter({'it': 234, 'he': 225, 'I': 160, 'He': 103, 'you': 100, 'they': 92, 'who': 89, 'him': 83, 'we': 64, 'them': 63, 'that': 63, 'It': 59, 'she': 59, 'her': 42, 'me': 40, 'She': 32, 'We': 26, 'us': 23, 'They': 22, 'himself': 21, 'You': 20, 'themselves': 14, 'itself': 8, 'myself': 7, 'herself': 6, 'yourself': 5, 'whom': 5, 'his': 2, 'Who': 2, 'yow': 2, 'ourselves': 2, "'im": 1, 'Whom': 1, "'em": 1, 'oneself': 1, 'hers': 1, 'ye': 1, 'thee': 1, 'thou': 1, 'yourselves': 1, 'Me': 1, 'mine': 1, 'ours': 1}), 'VERB': Counter({'is': 355, 'was': 307, 'be': 228, 'had': 196, 'are': 160, 'have': 145, 'were': 99, 'would': 91, 'been': 73, 'will': 71, 'said': 67, 'has': 66, 'could': 53, 'can': 51, 'may': 49, 'do': 49, 'must': 43, 'did': 42, 'should': 36, 'made': 35, 'make': 30, 'might': 26, 'know': 25, 'see': 24, 'came': 22, 'took': 21, 'take': 20, 'get': 20, 'used': 18, 'saw': 18, 'put': 18, 'come': 18, 'found': 18, 'United': 17, 'given': 17, 'to

In [7]:
print(emission_probabilities['get'])
print(words.index('get'))
# print(words.index('earthenware'))

Counter({'VERB': 0.003399158303658142, 'NUM': 0.002074688796680498, 'PRT': 0.0010741138560687433, 'CONJ': 0.0007457121551081282, 'PRON': 0.0005934718100890207, 'ADV': 0.0005271481286241434, 'ADJ': 0.0003436426116838488, '.': 0.0003370407819346141, 'DET': 0.00021616947686986597, 'ADP': 0.00019755037534571315, 'NOUN': 0.000106951871657754})
5020


###Evaluation
An example of evaluation:

In [8]:
# Viterbi Alorithm:
#   See the words in the sentence
#   List the possible tags for each sentence
#   Set d_i(1) for all of the pos for the first word as starting probability*emission probability
#   Select one pos of the second word->Find the d_i(2) for it by following max(d_i(1)*transition_probability(pos2|pos1)*emission_probability(word|pos2))
#   Store the pos1 which gave the max d_i(2).
#   Repeat this for all the pos tags of the second word i.e store all the d_i(2) and psi_i(2).
#   Repeat this for all the pos tags of the subsequent tags till the last word. Select the pos_last with the maximum d_i(last)
#   Bactrace from the last pos by choosin the psi_i from the last tag.

# See the words
test_sentence = [('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]

test_words = [tuple[0].lower() for tuple in test_sentence]

possible_tags = defaultdict(Counter)
for test_word in test_words:
  possible_tags[test_word] = [test_tag for test_tag in emission_probabilities[test_word] if emission_probabilities[test_word][test_tag] !=0]

print(test_words)
print(possible_tags)

['get', 'copper', 'or', 'earthenware', 'mugs', 'that', 'keep', 'beer', 'chilled', 'or', 'soup', 'hot', '.']
defaultdict(<class 'collections.Counter'>, {'get': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'copper': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'or': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'earthenware': [], 'mugs': [], 'that': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'keep': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'beer': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'chilled': [], 'soup': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], 'hot': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB'], '.': ['ADP', 'PRON', 'ADJ', 'ADV', '.', 'NOUN', 'NUM', 'CONJ', 'PRT', 'DET', 'VERB']})


In [9]:
from collections import defaultdict, Counter

def viterbi_algorithm(test_sentence, unique_tags, start_probabilities, transition_probabilities, emission_probabilities):
    delta_dict = defaultdict(lambda: Counter())
    back_pointer = defaultdict(lambda: Counter())

    first_word = test_sentence[0]
    for tag in unique_tags:
        delta_dict[first_word][tag] = start_probabilities.get(tag, 1e-12) * \
                                      emission_probabilities.get(first_word, defaultdict(lambda: 1e-12)).get(tag, 1e-12)

    for i in range(1, len(test_sentence)):
        word = test_sentence[i]
        for current_tag in unique_tags:
            best_prev_tag, max_prob = max(
                ((prev_tag, delta_dict[test_sentence[i-1]][prev_tag] *
                  transition_probabilities.get(current_tag, defaultdict(lambda: 1e-12)).get(prev_tag, 1e-12) *
                  emission_probabilities.get(word, defaultdict(lambda: 1e-12)).get(current_tag, 1e-12))
                 for prev_tag in unique_tags),
                key=lambda x: x[1]
            )
            delta_dict[word][current_tag] = max_prob
            back_pointer[word][current_tag] = best_prev_tag

    best_sequence = []
    last_word = test_sentence[-1]
    last_best_tag = max(delta_dict[last_word], key=delta_dict[last_word].get)
    best_sequence.append(last_best_tag)

    for i in range(len(test_sentence) - 2, -1, -1):
        last_best_tag = back_pointer[test_sentence[i+1]][last_best_tag]
        best_sequence.append(last_best_tag)

    return list(reversed(best_sequence))




In [10]:

sentence = [('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'),
            ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'),
            ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]


test_sentence = [word for word, _ in sentence]
true_tags = [tags for _,tags in sentence]


predicted_tags = viterbi_algorithm(test_sentence, unique_tags, start_probabilities, transition_probabilities, emission_probabilities)
print(true_tags)
print("Predicted POS Tags:", predicted_tags)
correct = 0
total = 0
correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
total += len(true_tags)
accuracy = correct / total

print(f"Accuracy: {accuracy * 100:.2f}%")

['VERB', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'CONJ', 'NOUN', 'ADJ', '.']
Predicted POS Tags: ['NOUN', '.', 'CONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', '.', 'CONJ', 'DET', 'ADJ', 'NOUN']
Accuracy: 38.46%


In [11]:
sentence = [('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')]
predicted_tags = ['DET', 'NOUN', 'CONJ', 'NOUN', 'ADP', 'PRON', 'VERB', 'NOUN', '.', 'CONJ', 'NOUN', 'ADJ', '.']
true_tags = ('VERB', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'PRON', 'VERB', 'NOUN', 'VERB', 'CONJ', 'NOUN', 'ADJ', '.')
correct = 0
total = 0
correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
total += len(true_tags)
accuracy = correct / total

print(f"Baseline Viterbi Accuracy: {accuracy * 100:.2f}%")
# print(f"Viterbi with Noise Handling Accuracy: {accuracy * 100:.2f}%")  # similarly calculate for Noise Handling


Baseline Viterbi Accuracy: 76.92%


In [12]:
print(test_data)

[[('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'CONJ'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')], [('By', 'ADP'), ('permitting', 'VERB'), ('freshman', 'NOUN'), ('students', 'NOUN'), ('we', 'PRON'), ('might', 'VERB'), ('extend', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('for', 'ADP'), ('such', 'PRT'), ('a', 'DET'), ('course', 'NOUN'), ('to', 'ADP'), ('some', 'DET'), ('individuals', 'NOUN'), ('who', 'PRON'), ('otherwise', 'ADV'), ('might', 'VERB'), ('never', 'ADV'), ('get', 'VERB'), ('to', 'PRT'), ('take', 'VERB'), ('it', 'PRON'), ('.', '.')], [('In', 'ADP'), ('stating', 'VERB'), ('this', 'DET'), ('position', 'NOUN'), (',', '.'), ('we', 'PRON'), ('should', 'VERB'), ('like', 'VERB'), ('to', 'PRT'), ('make', 'VERB'), ('it', 'PRON'), ('clear', 'ADJ'), ('to', 'ADP'), ('you', 'PRON'), ('that', 'ADP'), ('we', 'PRON'), ('cannot', 'VERB'), ('expect', 

In [13]:
test_accuracy = []

for sentence in test_data:
  test_sentence = [word.lower() for word, _ in sentence]
  true_tags = [tags for _,tags in sentence]

  predicted_tags = viterbi_algorithm(test_sentence, unique_tags, start_probabilities, transition_probabilities, emission_probabilities)
  # print(true_tags)
  # print("Predicted POS Tags:", predicted_tags)
  correct = 0
  total = 0
  correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
  total += len(true_tags)
  accuracy = correct / total
  test_accuracy.append(accuracy)

In [14]:
print(f"Mean Accuracy: {sum(test_accuracy)*100/len(test_accuracy):.2f}%")

Mean Accuracy: 75.11%


In [15]:
print(noisy_test_data)

[[('Get', 'VERB'), ('copper', 'NOUN'), ('or', 'UNK'), ('earthenware', 'NOUN'), ('mugs', 'NOUN'), ('that', 'PRON'), ('keep', 'VERB'), ('beer', 'NOUN'), ('chilled', 'VERB'), ('or', 'CONJ'), ('soup', 'NOUN'), ('hot', 'ADJ'), ('.', '.')], [('By', 'UNK'), ('permitting', 'VERB'), ('freshman', 'NOUN'), ('students', 'NOUN'), ('we', 'ADV'), ('might', 'VERB'), ('extend', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('for', 'ADP'), ('such', 'PRT'), ('a', 'DET'), ('course', 'NOUN'), ('to', 'ADP'), ('some', 'DET'), ('individuals', 'NOUN'), ('who', 'PRON'), ('otherwise', 'ADV'), ('might', 'ADV'), ('never', 'ADV'), ('get', 'VERB'), ('to', 'PRT'), ('take', 'VERB'), ('it', 'PRON'), ('.', '.')], [('In', 'UNK'), ('stating', 'VERB'), ('this', 'DET'), ('position', 'NOUN'), (',', '.'), ('we', 'PRON'), ('should', 'VERB'), ('like', 'VERB'), ('to', 'PRT'), ('make', 'VERB'), ('it', 'PRON'), ('clear', 'ADJ'), ('to', 'ADP'), ('you', 'PRON'), ('that', 'ADP'), ('we', 'PRON'), ('cannot', 'VERB'), ('expect', 'VE

In [19]:

noisy_test_accuracy = []

for sentence in noisy_test_data:
  test_sentence = [word.lower().strip() for word, _ in sentence]
  true_tags = [tags for _,tags in sentence]


  predicted_tags = viterbi_algorithm(test_sentence, unique_tags, start_probabilities, transition_probabilities, emission_probabilities)
  # print(true_tags)
  # print("Predicted POS Tags:", predicted_tags)
  correct = 0
  total = 0
  correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
  total += len(true_tags)
  accuracy = correct / total
  noisy_test_accuracy.append(accuracy)
print(f"Mean Accuracy: {sum(noisy_test_accuracy)*100/len(noisy_test_accuracy):.2f}%")

Mean Accuracy: 68.18%
